In [1]:
import numpy as np
import pandas as pd

# 0. Inputs

In [2]:
D = 6  # tank diameter (ft)
H = 12  # tank height (ft)
?? = 8  # hold ? ft of product
paint_color = 'white'
liq_level_high = 11.5
liq_level_low = 4.5
liq_flow = 8450  # (gal/yr)

{
    'Vertical Cylinder',
    'Horizontal Cylinder',
    'External Floating Roof Tank',
    'Internal Floatign Roof Tank',
    'Domed Floating Roof Tank',
}

Object ` = 8  # hold ? ft of product` not found.


In [12]:
def calc_Vv(D, Hvo):
    """
    Vv = Tank vapor space volume (ft^3, Eq 1-3)
    D = Tank diameter (ft)
    Hvo = vapor space outage (ft, Eq 1-16)
    """
    return (np.pi / 4) * D ** 2 * Hvo

In [ ]:
def calc_Hvo(Hs, Hl, Hro):
    """
    Hs = Tank shell height (ft, given)
    Hl  = Tank liquid height (ft, given, assumed to be half-filled mostly)
    Hro = roof outage (ft, Eq 1-17)
    """
    return Hs - Hl + Hro

# 1. Utilities

In [2]:
def calc_F_to_R(F):
    return F + 459.67

def calc_R_to_F(R):
    return R - 459.67

def calc_gal_to_bbl(gal):
    return gal / 42

def calc_bbl_to_gal(bbl):
    return bbl * 42

# 2. Standing Loss

In [ ]:
# Fix roof tank



In [3]:

def calc_T_LA(Hs, D, Ta, Tb, alpa_r, alpha_s, I):
    _1st = (0.5 - 0.8 / (4.4 * Hs/D + 3.8)) * Ta
    _2nd = (0.5 + 0.8 / (4.4 * Hs/D + 3.8)) * Tb
    _3rd = (0.021 * alpha_r * I + 0.013 * Hs/D * alpha_s * I) / (4.4 * Hs/D + 3.8)
    return _1st + _2nd + _3rd 

In [4]:
# December
Hs = 30
D = 15
Ta = calc_F_to_R(95)  # max, Cedar City, Utah
Tb = calc_F_to_R(110)   # liquid bulk
alpha_r = 0.49          # tan
alpha_s = 0.49          # tan
I = 741

In [5]:
calc_T_LA(Hs, D, Ta, Tb, alpha_r, alpha_s, I)

564.4767642857144

## Geological data retrieval

In [4]:
import pandas as pd
from fuzzywuzzy import process

C:\Users\EricKim\AppData\Roaming\Python\Python39\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [9]:
file = 'Table 7-1-7 Meteorological data for selected US locations.xlsx'
df = pd.read_excel(file, sheet_name='Data')

# Average daily ambient temperature (Eq. 1-30)
df_tav = df[df['Symbol'].isin(['TAN', 'TAX'])].groupby(['Location', 'City', 'State', 'Units']).mean().reset_index()
df_tav['Symbol'] = 'TAA'
df = pd.concat([df, df_tav], ignore_index=True)

input_location = 'Cedar'
locations = set(df['Location'])
if input_location not in locations:
    closest_match, score = process.extractOne(input_location, locations)
    raise TypeError('Invalid location parameter: "%s". Did you mean "%s"?' % (input_location, closest_match))

# "Location == 'Cedar City, UT' and Symbol == 'TAN' and Units == 'mi/hr'"
location_data = df[(df['Location'] == input_location)]

T_max = location_data[location_data['Symbol'] == 'TAX']
T_min = location_data[location_data['Symbol'] == 'TAN']
T_avg = location_data[location_data['Symbol'] == 'TAA']
V = location_data[location_data['Symbol'] == 'V']
I = location_data[location_data['Symbol'] == 'I']
PA = location_data[location_data['Symbol'] == 'PA']

TypeError: Invalid location parameter: "Cedar". Did you mean "Cedar City, UT"?

In [10]:
location_data

,Location,City,State,Symbol,Units,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
1065,"Cedar City, UT",Cedar City,UT,TAN,°F,19.20,23.30,28.30,33.5,42.20,50.9,59.00,57.70,47.7,36.00,25.00,18.7,36.80
1066,"Cedar City, UT",Cedar City,UT,TAX,°F,41.90,46.00,54.60,61.1,72.30,82.5,89.50,87.20,79.3,65.70,51.70,41.5,64.40
1067,"Cedar City, UT",Cedar City,UT,V,mi/hr,6.30,6.70,8.10,8.9,8.70,8.7,7.60,7.60,7.2,6.70,6.30,6.3,7.40
1068,"Cedar City, UT",Cedar City,UT,I,Btu/ft2/day,818.00,1072.00,1548.00,1969.0,2299.00,2534.0,2285.00,2082.00,1827.0,1355.00,944.00,741.0,1623.00
1069,"Cedar City, UT",Cedar City,UT,PA,lb/in2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.01
1248,"Cedar City, UT",Cedar City,UT,TAA,°F,30.55,34.65,41.45,47.3,57.25,66.7,74.25,72.45,63.5,50.85,38.35,30.1,50.60
